# Explain Python Model Smoke Tests

This notebook validates that `ModelEngine` can:
- Load model definitions from `definitions/*.json`
- Instantiate all configured models
- Run one simulation step without errors

In [4]:
from pathlib import Path
from model_engine import ModelEngine

repo_root = Path.cwd()
definitions_dir = repo_root / "definitions"

print("Repo root:", repo_root)
print("Definitions dir exists:", definitions_dir.exists())

Repo root: /Users/timantonius/Projects/explain/explain-python
Definitions dir exists: True


In [5]:
definition_files = [
    "baseline.json",
    "aop.json",
    "aph.json",
    "mas.json",
    "pda.json",
    "example_minimal.json",
]

single_results = {}
for file_name in definition_files:
    engine = ModelEngine().load_json_file(definitions_dir / file_name)
    engine.step_model()
    single_results[file_name] = {
        "initialized": engine.is_initialized,
        "model_count": len(engine.models),
        "model_names": sorted(engine.models.keys()),
        "stepsize": engine.modeling_stepsize,
    }

single_results

{'baseline.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['AA', 'SYS_RES', 'VEN'],
  'stepsize': 0.0005},
 'aop.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['AA', 'SYS_RES', 'VEN'],
  'stepsize': 0.0005},
 'aph.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['AA', 'SYS_RES', 'VEN'],
  'stepsize': 0.0005},
 'mas.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['AA', 'SYS_RES', 'VEN'],
  'stepsize': 0.0005},
 'pda.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['AA', 'SYS_RES', 'VEN'],
  'stepsize': 0.0005},
 'example_minimal.json': {'initialized': True,
  'model_count': 3,
  'model_names': ['C1', 'C2', 'R1'],
  'stepsize': 0.01}}

In [3]:
failed = [
    name
    for name, result in single_results.items()
    if (not result["initialized"]) or result["model_count"] == 0
]

if failed:
    raise AssertionError(f"Smoke test failed for: {failed}")

print("All model definitions loaded and stepped successfully.")

All model definitions loaded and stepped successfully.
